In [1]:
#ignore
#!pip install pyAudioAnalysis

In [2]:
#ignore
#!pip install ffmpeg

In [3]:
#ignore
#!pip install python-magic

In [4]:
#will actually need this
#!pip install librosa

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy
import matplotlib.pyplot as plt
import librosa
#from pyAudioAnalysis import audioBasicIO #don't really need pyaudioanalysis
#from pyAudioAnalysis import ShortTermFeatures

C:\Users\peter\Anaconda3\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\peter\Anaconda3\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


covariance:

6 combinations:
X X
X Y
X Z
Y Y
Z Z
Y Z

Start Here

In [6]:
original = pd.read_pickle('joined.pkl')

In [7]:
ids = original.pid.unique()

dummy = original[original['pid'] == ids[0]]

df = original

In [8]:
#51 rows ~ 1 second, where time is in milliseconds

print(dummy['time'].iloc[0])
print(dummy['time'].iloc[50])

print(dummy['time'].iloc[51])
print(dummy['time'].iloc[101])

print(dummy['time'].iloc[102])
print(dummy['time'].iloc[152])

print(dummy['time'].iloc[153])
print(dummy['time'].iloc[203])

1493739732016
1493739733023
1493739733043
1493739734049
1493739734070
1493739735083
1493739735103
1493739736097


In [9]:
def getZeroCrossingRate(arr):
        my_array = np.array(arr)
        return float("{0:.2f}".format((((my_array[:-1] * my_array[1:]) < 0).sum())/len(arr)))

In [10]:
def fft_and_zero_cross(df, pid):
    
    sec = 51
    short = sec*2
    full = sec*10
    big = sec*1000
    
    df = df[df['pid'] == pid].copy().reset_index().drop('index', axis = 1)
    
    windows = np.arange(0,len(df),big)
    
    
    df['x_fft_1'] = 0
    df['x_fft_2'] = 0
    df['x_fft_3'] = 0

    df['y_fft_1'] = 0
    df['y_fft_2'] = 0
    df['y_fft_3'] = 0
    
    df['z_fft_1'] = 0
    df['z_fft_2'] = 0
    df['z_fft_3'] = 0

    df_app = pd.DataFrame(columns = df.columns)
    
    for i in range(len(windows)-1):
        
        small = df.iloc[windows[i]:windows[i+1]].copy()
        
        x_fft = scipy.fft.fft(np.array(small['x'])).real
        
        small['x_fft_1'] = x_fft[0]
        small['x_fft_2'] = x_fft[1]
        small['x_fft_3'] = x_fft[2]
        small['x_fft_max'] = x_fft.max()
        
        small['x_zcr'] = getZeroCrossingRate(np.array(small['x']))
        
        y_fft = scipy.fft.fft(np.array(small['y'])).real
        
        small['y_fft_1'] = y_fft[0]
        small['y_fft_2'] = y_fft[1]
        small['y_fft_3'] = y_fft[2]
        small['y_fft_max'] = y_fft.max()
        
        small['y_zcr'] = getZeroCrossingRate(np.array(small['y']))
        
        z_fft = scipy.fft.fft(np.array(small['z'])).real
        
        small['z_fft_1'] = z_fft[0]
        small['z_fft_2'] = z_fft[1]
        small['z_fft_3'] = z_fft[2]
        small['z_fft_max'] = z_fft.max()
        
        small['z_zcr'] = getZeroCrossingRate(np.array(small['z']))
        
        df_app = df_app.append(small)
        
    df = df_app
    
    return df

In [11]:
def mfcc(df, pid):
    
    #give whole df, one pid at a time
    
    df = df[df['pid'] == pid].copy().reset_index().drop('index', axis = 1)
    
    Fs = 40
    mfcc_x_array = librosa.feature.mfcc(np.array(df['x']), sr=Fs).T
    mfcc_y_array = librosa.feature.mfcc(np.array(df['y']), sr=Fs).T
    mfcc_z_array = librosa.feature.mfcc(np.array(df['z']), sr=Fs).T

    x_mfcc = pd.DataFrame(mfcc_x_array)
    y_mfcc = pd.DataFrame(mfcc_y_array)
    z_mfcc = pd.DataFrame(mfcc_z_array)

    expansion_x = round(len(df)/len(x_mfcc))
    new_index_x = (pd.DataFrame(x_mfcc).index * expansion_x)
    x_mfcc.index = new_index_x

    expansion_y = round(len(df)/len(y_mfcc))
    new_index_y = (pd.DataFrame(y_mfcc).index * expansion_y)
    y_mfcc.index = new_index_x

    expansion_z = round(len(df)/len(z_mfcc))
    new_index_z = (pd.DataFrame(z_mfcc).index * expansion_z)
    z_mfcc.index = new_index_z

    df = pd.merge_asof(df,x_mfcc, left_index = True, right_index = True)
    df = pd.merge_asof(df,y_mfcc, left_index = True, right_index = True)
    df = pd.merge_asof(df,z_mfcc, left_index = True, right_index = True)
 
    #new_names = [(i,i+'_z') for i in df.iloc[:, 6:].columns.values]
    #df.rename(columns = dict(new_names), inplace=True)

    return df

In [12]:
def rms(df,pid):
    
    #give whole df, one pid at a time
    
    df = df[df['pid'] == pid].copy()
    
    sec = 51
    short = sec*2
    full = sec*10
    
    df['x_rms'] = ((df['x']**2)**.5).rolling(window=short).mean()
    df['y_rms'] = ((df['y']**2)**.5).rolling(window=short).mean()
    df['z_rms'] = ((df['z']**2)**.5).rolling(window=short).mean()
    
    return df

In [13]:
def basic(df,pid):
     
    sec = 51
    short = sec*2
    full = sec*10
    
    df = df[df['pid'] == pid].copy()
    
    df['x_mean_short'] = df['x'].rolling(window=short).mean()
    df['x_std_short'] = df['x'].rolling(window=short).std()
    df['x_mean_long'] = df['x'].rolling(window=full).mean()
    df['x_std_long'] = df['x'].rolling(window=full).std()
    
    df['y_mean_short'] = df['y'].rolling(window=short).mean()
    df['y_std_short'] = df['y'].rolling(window=short).std()
    df['y_mean_long'] = df['y'].rolling(window=full).mean()
    df['y_std_long'] = df['y'].rolling(window=full).std()
                          
    df['z_mean_short'] = df['z'].rolling(window=short).mean()
    df['z_std_short'] = df['z'].rolling(window=short).std()
    df['z_mean_long'] = df['z'].rolling(window=full).mean()
    df['z_std_long'] = df['z'].rolling(window=full).std()
    
    return df

In [14]:
def extract(df,pid,feature):
    
    #give this x, y, z, for each pid
    
    sec = 51
    short = sec*2
    full = sec*10
    
    df = df[df['pid'] == pid].copy()
    
    df[feature + '_mean'] = df[feature].rolling(window=full).mean()
    df[feature + '_median'] = df[feature].rolling(window=full).median()
    df[feature + '_std'] = df[feature].rolling(window=full).std()
    
    df = summarize(df,pid,feature + '_mean')
    df = summarize(df,pid,feature + '_median')
    df = summarize(df,pid,feature + '_std')
    
    return df

In [15]:
def summarize(df,pid,feature):
    
    #called by extract, don't call separately
    
    #feature: mean over rolling window of 10 seconds
    #break into windows of 2 seconds
    #take mean, variance, max, min of those 5 windows as new features
    #take mean of bottom 2 seconds and mean of top 2 seconds as new features
    
    df = df[df['pid'] == pid].copy()
    
    sec = 51
    short = sec*2
    full = sec*10
    
    df[feature + '_first'] = df[feature].shift(short*4).rolling(window=short).mean()
    df['second'] = df[feature].shift(short*3).rolling(window=short).mean()
    df['third'] = df[feature].shift(short*2).rolling(window=short).mean()
    df['fourth'] = df[feature].shift(short*1).rolling(window=short).mean()
    df[feature + '_fifth'] = df[feature].rolling(window=short).mean()
    
    df[feature + '_mean'] = df[[feature + '_first', 'second', 
                                'third', 'fourth', feature + '_fifth']].mean(axis=1)
    
    df[feature + '_var'] = df[[feature + '_first', 'second', 
                                'third', 'fourth', feature + '_fifth']].var(axis=1)
    
    df[feature + '_max'] = df[[feature + '_first', 'second', 
                                'third', 'fourth', feature + '_fifth']].max(axis=1)
    
    df[feature + '_min'] = df[[feature + '_first', 'second', 
                                'third', 'fourth', feature + '_fifth']].min(axis=1)
    
    df.drop(['second','third','fourth'],axis = 1, inplace = True)
    
    df[feature + '_first' + '_diff'] = df[feature + '_first'].diff(periods = full)
    df[feature + '_fifth' + '_diff'] = df[feature + '_fifth'].diff(periods = full)
    df[feature + '_mean' + '_diff'] = df[feature + '_mean'].diff(periods = full)
    df[feature + '_var' + '_diff'] = df[feature + '_var'].diff(periods = full)
    df[feature + '_max' + '_diff'] = df[feature + '_max'].diff(periods = full)
    df[feature + '_min' + '_diff'] = df[feature + '_max'].diff(periods = full)

    return df

In [16]:
#calculate 3 fft components and max, and zero crossing rate for each pid

start = datetime.now()

fz_df = pd.DataFrame(columns = (fft_and_zero_cross(df, ids[0])).columns)

for i in range(len(ids)):
    fz_df = fz_df.append(fft_and_zero_cross(df,ids[i]))
    
df = fz_df
    
end = datetime.now()

end - start

datetime.timedelta(seconds=77, microseconds=953226)

In [17]:
#calculate mfccs for each pid

start = datetime.now()

mfcc_df = pd.DataFrame(columns = (mfcc(df, ids[0])).columns)

for i in range(len(ids)):
    mfcc_df = mfcc_df.append(mfcc(df,ids[i]))
    
df = mfcc_df
    
end = datetime.now()

end - start

datetime.timedelta(seconds=86, microseconds=426366)

In [18]:
#calculate rms for x, y, and z for each pid

start = datetime.now()

rms_df = pd.DataFrame(columns = (rms(df, ids[0])).columns)

for i in range(len(ids)):
    rms_df = rms_df.append(rms(df,ids[i]))
    
df = rms_df
    
end = datetime.now()

end - start

datetime.timedelta(seconds=130, microseconds=70697)

In [19]:
#calculate rolling mean and std for x, y, and z for each pid

start = datetime.now()

basic_df = pd.DataFrame(columns = (basic(df, ids[0])).columns)

for i in range(len(ids)):
    basic_df = basic_df.append(basic(df,ids[i]))
    
df = basic_df
    
end = datetime.now()

end - start

datetime.timedelta(seconds=221, microseconds=434334)

In [20]:
#comes out to ~7 gigs

start = datetime.now()

df.to_pickle('./processed.pkl')

end = datetime.now()

end - start

datetime.timedelta(seconds=74, microseconds=504682)